# Preparar el fichero `orders_data.parquet` de modo que pueda ser usado para contruir un 'forecasting model'.


In [0]:
from pyspark.sql import (
    SparkSession,
    types,
    functions as F,
)

spark = (
    SparkSession
    .builder
    .appName('cleaning_orders_dataset_with_pyspark')
    .getOrCreate()
)

In [0]:
df = spark.read.parquet('dbfs:/FileStore/orders_data.parquet')
df.toPandas().head()

,order_date,order_id,product,product_id,category,purchase_address,quantity_ordered,price_each,cost_price,turnover,margin
0,2023-01-22 21:25:00,141234,iPhone,5.638009e+12,Vêtements,"944 Walnut St, Boston, MA 02215",1,700.00,231.0000,700.00,469.0000
1,2023-01-28 14:15:00,141235,Lightning Charging Cable,5.563320e+12,Alimentation,"185 Maple St, Portland, OR 97035",1,14.95,7.4750,14.95,7.4750
2,2023-01-17 13:33:00,141236,Wired Headphones,2.113973e+12,Vêtements,"538 Adams St, San Francisco, CA 94016",2,11.99,5.9950,23.98,11.9900
3,2023-01-05 20:33:00,141237,27in FHD Monitor,3.069157e+12,Sports,"738 10th St, Los Angeles, CA 90001",1,149.99,97.4935,149.99,52.4965
4,2023-01-25 11:59:00,141238,Wired Headphones,9.692681e+12,Électronique,"387 10th St, Austin, TX 73301",1,11.99,5.9950,11.99,5.9950


## 1. Modify: Remove orders placed between 12am and 5am (inclusive); convert from timestamp to date 

- Eliminar las órdenes realizadas entre las 12am y las 5am (inclusive)
- Convertir la columna order_date de formato timestamp a solo fecha

In [0]:
# Filtrar para eliminar órdenes realizadas entre las 12am y 5am (inclusive)
filtered_df = df.filter(~((F.hour('order_date') >= 0) & (F.hour('order_date') <= 5)))

# Convertir el timestamp a solo fecha
cleaned_df = filtered_df.withColumn('order_date', F.to_date('order_date'))

cleaned_df.toPandas().head()

,order_date,order_id,product,product_id,category,purchase_address,quantity_ordered,price_each,cost_price,turnover,margin
0,2023-01-22,141234,iPhone,5.638009e+12,Vêtements,"944 Walnut St, Boston, MA 02215",1,700.00,231.0000,700.00,469.0000
1,2023-01-28,141235,Lightning Charging Cable,5.563320e+12,Alimentation,"185 Maple St, Portland, OR 97035",1,14.95,7.4750,14.95,7.4750
2,2023-01-17,141236,Wired Headphones,2.113973e+12,Vêtements,"538 Adams St, San Francisco, CA 94016",2,11.99,5.9950,23.98,11.9900
3,2023-01-05,141237,27in FHD Monitor,3.069157e+12,Sports,"738 10th St, Los Angeles, CA 90001",1,149.99,97.4935,149.99,52.4965
4,2023-01-25,141238,Wired Headphones,9.692681e+12,Électronique,"387 10th St, Austin, TX 73301",1,11.99,5.9950,11.99,5.9950


## 2. New column containing (lower bound inclusive, upper bound exclusive): "morning" for orders placed 5-12am, "afternoon" for orders placed 12-6pm, and "evening" for 6-12pm

Como ya convertimos order_date a una columna de solo fecha en el paso anterior, me daba error asi que voy a trabajar directamente con la columna original order_date antes de convertirla a fecha o creando una columna temporal con la hora antes de la conversión.

In [0]:
# Crear una columna temporal con la hora extraída antes de convertir a fecha
temp_df = df.withColumn('hour', F.hour('order_date'))

# Filtrar las órdenes entre 12am y 5am, y luego convertir a fecha
filtered_df = temp_df.filter(~((F.col('hour') >= 0) & (F.col('hour') <= 5)))

# Clasificar las órdenes según el período del día
classified_df = filtered_df.withColumn(
    'time_of_day',
    F.when((F.col('hour') >= 5) & (F.col('hour') < 12), 'morning')
     .when((F.col('hour') >= 12) & (F.col('hour') < 18), 'afternoon')
     .when((F.col('hour') >= 18) & (F.col('hour') < 24), 'evening')
     .otherwise('unknown')
)

# Luego ya se vuelve a convertir `order_date` a solo fecha
final_df = classified_df.withColumn('order_date', F.to_date('order_date')).drop('hour')

final_df.toPandas().head()
# final_df.select('product').distinct().show(20, truncate=False)


,order_date,order_id,product,product_id,category,purchase_address,quantity_ordered,price_each,cost_price,turnover,margin,time_of_day
0,2023-01-22,141234,iPhone,5.638009e+12,Vêtements,"944 Walnut St, Boston, MA 02215",1,700.00,231.0000,700.00,469.0000,evening
1,2023-01-28,141235,Lightning Charging Cable,5.563320e+12,Alimentation,"185 Maple St, Portland, OR 97035",1,14.95,7.4750,14.95,7.4750,afternoon
2,2023-01-17,141236,Wired Headphones,2.113973e+12,Vêtements,"538 Adams St, San Francisco, CA 94016",2,11.99,5.9950,23.98,11.9900,afternoon
3,2023-01-05,141237,27in FHD Monitor,3.069157e+12,Sports,"738 10th St, Los Angeles, CA 90001",1,149.99,97.4935,149.99,52.4965,evening
4,2023-01-25,141238,Wired Headphones,9.692681e+12,Électronique,"387 10th St, Austin, TX 73301",1,11.99,5.9950,11.99,5.9950,morning


## 3. Remove rows containing "TV" as the company has stopped selling this product; ensure all values are lowercase



- Convertir todos los valores de la columna product a minúsculas.
- Eliminar las filas donde el producto contenga "TV".

> En la ultima linea del anterior ejercicio hay comentada una linea en la que muestro los 20 primeros registros para comprobar si hay algun Tv, efectivamente aparece un tal "Flatscreen TV". 

> Al ejecutar nuestro codigo de este erjecicio podemos ver que ese registro ya no aparece y que se encuentra todo en minusculas. De momento dejo para que se ejecute el pandas pero puedes descomentar la linea de abajo y comprobar esto tu mismo. 

In [0]:
# Convertir a minúsculas
lowercase_df = final_df.withColumn('product', F.lower(F.col('product')))

# Filtrar para eliminar filas donde el producto contenga "tv"
# Aqui como ya todo esta en minusculas se deja entre las comillas tv en minuscula, no hace falta mas
filtered_df = lowercase_df.filter(~F.col('product').contains('tv'))

filtered_df.toPandas().head()
# filtered_df.select('product').distinct().show(20, truncate=False)


,order_date,order_id,product,product_id,category,purchase_address,quantity_ordered,price_each,cost_price,turnover,margin,time_of_day
0,2023-01-22,141234,iphone,5.638009e+12,Vêtements,"944 Walnut St, Boston, MA 02215",1,700.00,231.0000,700.00,469.0000,evening
1,2023-01-28,141235,lightning charging cable,5.563320e+12,Alimentation,"185 Maple St, Portland, OR 97035",1,14.95,7.4750,14.95,7.4750,afternoon
2,2023-01-17,141236,wired headphones,2.113973e+12,Vêtements,"538 Adams St, San Francisco, CA 94016",2,11.99,5.9950,23.98,11.9900,afternoon
3,2023-01-05,141237,27in fhd monitor,3.069157e+12,Sports,"738 10th St, Los Angeles, CA 90001",1,149.99,97.4935,149.99,52.4965,evening
4,2023-01-25,141238,wired headphones,9.692681e+12,Électronique,"387 10th St, Austin, TX 73301",1,11.99,5.9950,11.99,5.9950,morning


## 4. Ensure all values are lowercase

> Aqui como la columna que cree ya va en minusculas, los productos tambien y muchas son numericas, solo habria que transformar la d `category` y la de d `purchase_address` como lo hemos hecho antes

In [0]:
# Convertir todas las columnas de tipo texto a minúsculas
lowercase_df = filtered_df.withColumn('category', F.lower(F.col('category'))) \
                          .withColumn('purchase_address', F.lower(F.col('purchase_address')))

lowercase_df.toPandas().head()


,order_date,order_id,product,product_id,category,purchase_address,quantity_ordered,price_each,cost_price,turnover,margin,time_of_day
0,2023-01-22,141234,iphone,5.638009e+12,vêtements,"944 walnut st, boston, ma 02215",1,700.00,231.0000,700.00,469.0000,evening
1,2023-01-28,141235,lightning charging cable,5.563320e+12,alimentation,"185 maple st, portland, or 97035",1,14.95,7.4750,14.95,7.4750,afternoon
2,2023-01-17,141236,wired headphones,2.113973e+12,vêtements,"538 adams st, san francisco, ca 94016",2,11.99,5.9950,23.98,11.9900,afternoon
3,2023-01-05,141237,27in fhd monitor,3.069157e+12,sports,"738 10th st, los angeles, ca 90001",1,149.99,97.4935,149.99,52.4965,evening
4,2023-01-25,141238,wired headphones,9.692681e+12,électronique,"387 10th st, austin, tx 73301",1,11.99,5.9950,11.99,5.9950,morning


## 5. New column containing: the State that the purchase was ordered from

> Las direcciones siguen todas el mismo formato, por lo que veo el estado es el que esta despues de la coma, siendo las dos letras esas. 
Ej: ma = Massachusetts

> Entonces en la columna que se genere van a quedar las dos letras, el nombre del estado como tal pues no, para eso ya le puedes meter mas tablas en el futuro que los compare y filtre



In [0]:
state_extracted_df = lowercase_df.withColumn(
    'purchase_state',
    F.split(F.col('purchase_address'), ', ')[2].substr(1, 2)  
)

state_extracted_df.toPandas().head()


,order_date,order_id,product,product_id,category,purchase_address,quantity_ordered,price_each,cost_price,turnover,margin,time_of_day,purchase_state
0,2023-01-22,141234,iphone,5.638009e+12,vêtements,"944 walnut st, boston, ma 02215",1,700.00,231.0000,700.00,469.0000,evening,ma
1,2023-01-28,141235,lightning charging cable,5.563320e+12,alimentation,"185 maple st, portland, or 97035",1,14.95,7.4750,14.95,7.4750,afternoon,or
2,2023-01-17,141236,wired headphones,2.113973e+12,vêtements,"538 adams st, san francisco, ca 94016",2,11.99,5.9950,23.98,11.9900,afternoon,ca
3,2023-01-05,141237,27in fhd monitor,3.069157e+12,sports,"738 10th st, los angeles, ca 90001",1,149.99,97.4935,149.99,52.4965,evening,ca
4,2023-01-25,141238,wired headphones,9.692681e+12,électronique,"387 10th st, austin, tx 73301",1,11.99,5.9950,11.99,5.9950,morning,tx


Este codigo te le explico un poco mas porque la parte del split puede ser un poco liosa. Esto funciona y se puede hacer asi porque todas las direcciones llevan el mismo formato, sino habria que limpiar eso tmb

> Aqui se usa el split que nos separa el campo en partes, en algo como asi `['944 walnut st', 'boston', 'ma 02215']`, lo divide segun las comas. 

>Luego con el `(2)` tenemos indicado que esta el estado en la tercera division, que esto empieza a contar en cero. 

> Y luego el `.substr(1, 2)` para pillar solo los dos primeros caracteres de esta parte que esos ya son los códigos de estado

Se acaba de indicar que el estado este preferblemente en mayusculas asiq edito el codigo aqui para que la nueva columna tenga las dos letras en mayucula a pesar de ser extraidas en minuscula

In [0]:
# Extraer el estado del campo `purchase_address` y convertirlo a mayúsculas
state_extracted_df = lowercase_df.withColumn(
    'purchase_state',
    F.upper(F.split(F.col('purchase_address'), ', ')[2].substr(1, 2))  # Convertir a mayúsculas
)

state_extracted_df.toPandas().head()

,order_date,order_id,product,product_id,category,purchase_address,quantity_ordered,price_each,cost_price,turnover,margin,time_of_day,purchase_state
0,2023-01-22,141234,iphone,5.638009e+12,vêtements,"944 walnut st, boston, ma 02215",1,700.00,231.0000,700.00,469.0000,evening,MA
1,2023-01-28,141235,lightning charging cable,5.563320e+12,alimentation,"185 maple st, portland, or 97035",1,14.95,7.4750,14.95,7.4750,afternoon,OR
2,2023-01-17,141236,wired headphones,2.113973e+12,vêtements,"538 adams st, san francisco, ca 94016",2,11.99,5.9950,23.98,11.9900,afternoon,CA
3,2023-01-05,141237,27in fhd monitor,3.069157e+12,sports,"738 10th st, los angeles, ca 90001",1,149.99,97.4935,149.99,52.4965,evening,CA
4,2023-01-25,141238,wired headphones,9.692681e+12,électronique,"387 10th st, austin, tx 73301",1,11.99,5.9950,11.99,5.9950,morning,TX


## 6. Guardar archivo final limpio con nombre `orders_data_clean.parquet`

In [0]:
state_extracted_df.write.mode('overwrite').parquet('dbfs:/FileStore/orders_data_clean.parquet')

Comprobamos que el archivo se haya guardado bien por si acaso 


----->

In [0]:
test_df = spark.read.parquet('dbfs:/FileStore/orders_data_clean.parquet')
test_csv_pandas_df = test_df.toPandas()

test_csv_pandas_df.head()

,order_date,order_id,product,product_id,category,purchase_address,quantity_ordered,price_each,cost_price,turnover,margin,time_of_day,purchase_state
0,2023-01-22,141234,iphone,5.638009e+12,vêtements,"944 walnut st, boston, ma 02215",1,700.00,231.0000,700.00,469.0000,evening,MA
1,2023-01-28,141235,lightning charging cable,5.563320e+12,alimentation,"185 maple st, portland, or 97035",1,14.95,7.4750,14.95,7.4750,afternoon,OR
2,2023-01-17,141236,wired headphones,2.113973e+12,vêtements,"538 adams st, san francisco, ca 94016",2,11.99,5.9950,23.98,11.9900,afternoon,CA
3,2023-01-05,141237,27in fhd monitor,3.069157e+12,sports,"738 10th st, los angeles, ca 90001",1,149.99,97.4935,149.99,52.4965,evening,CA
4,2023-01-25,141238,wired headphones,9.692681e+12,électronique,"387 10th st, austin, tx 73301",1,11.99,5.9950,11.99,5.9950,morning,TX


## 7. Exportar archivo limpio en formato CSV 

In [0]:
# Guardar el DataFrame final como un archivo CSV
state_extracted_df.write.mode('overwrite') \
    .option('header', True) \
    .csv('dbfs:/FileStore/orders_data_clean.csv')

Comprobamos que el archivo se haya guardado bien por si acaso 

----->

In [0]:
test_csv_df = spark.read.csv('dbfs:/FileStore/orders_data_clean.csv', header=True, inferSchema=True)
test_csv_pandas_df = test_csv_df.toPandas()

test_csv_pandas_df.head()


,order_date,order_id,product,product_id,category,purchase_address,quantity_ordered,price_each,cost_price,turnover,margin,time_of_day,purchase_state
0,2023-01-22,141234,iphone,5.638009e+12,vêtements,"944 walnut st, boston, ma 02215",1,700.00,231.0000,700.00,469.0000,evening,MA
1,2023-01-28,141235,lightning charging cable,5.563320e+12,alimentation,"185 maple st, portland, or 97035",1,14.95,7.4750,14.95,7.4750,afternoon,OR
2,2023-01-17,141236,wired headphones,2.113973e+12,vêtements,"538 adams st, san francisco, ca 94016",2,11.99,5.9950,23.98,11.9900,afternoon,CA
3,2023-01-05,141237,27in fhd monitor,3.069157e+12,sports,"738 10th st, los angeles, ca 90001",1,149.99,97.4935,149.99,52.4965,evening,CA
4,2023-01-25,141238,wired headphones,9.692681e+12,électronique,"387 10th st, austin, tx 73301",1,11.99,5.9950,11.99,5.9950,morning,TX
